In [2]:
import sys
sys.path.insert(0, r'C:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages')

import optuna
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dropout
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

# Cargar los datos 
ruta = r'C:/Users/Administrator/Documents/Proyecto seg clientes refuerzo/Proyecto_seg_clientes/Data/teleCust1000t_listo.csv'


df = pd.read_csv(ruta)

# Ver las primeras filas
print(df.head())
print(df.info())

# Separar las características (X) y la variable objetivo (y)
X = df.drop('custcat', axis=1)  
y = df['custcat']

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Verificar el balance después de SMOTE
print("Distribución después de SMOTE:")
print(pd.Series(y_resampled).value_counts())

# Dividir el dataset en conjunto de entrenamiento, validación y test
X_train, X_temp, y_train, y_temp = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Codificar la variable objetivo (One-Hot Encoding)
y_train_encoded = to_categorical(y_train - 1)  # Ajustar para que las clases empiecen en 0
y_val_encoded = to_categorical(y_val - 1)
y_test_encoded = to_categorical(y_test - 1)



# Definir el EarlyStopping como variable global
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def create_model(trial):
    # Optimización de hiperparámetros
    # Definir el número de neuronas por capa
    num_neurons_1 = trial.suggest_int('num_neurons_1', 32, 128)
    num_neurons_2 = trial.suggest_int('num_neurons_2', 16, 64)
    
    # Regularización L2
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-2)
    
    # Tasa de aprendizaje
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    # Crear el modelo
    model = Sequential([
        Dense(num_neurons_1, activation='elu', kernel_regularizer=l2(l2_reg), input_dim=X_train_scaled.shape[1]),
        Dense(num_neurons_2, activation='elu', kernel_regularizer=l2(l2_reg)),
        Dense(y_train_encoded.shape[1], activation='softmax')
    ])
    
    # Compilar el modelo
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

def objective(trial):
    # Crear el modelo con los hiperparámetros sugeridos
    model = create_model(trial)
    
    # Entrenar el modelo
    history = model.fit(
        X_train_scaled, y_train_encoded,
        epochs=200,
        batch_size=8,
        validation_data=(X_val_scaled, y_val_encoded),
        callbacks=[early_stopping],
        verbose=0
    )
    
    # Obtener la exactitud en el conjunto de validación
    val_accuracy = history.history['val_accuracy'][-1]
    
    return val_accuracy  # Maximizar la exactitud en validación

# Crear el estudio de Optuna
study = optuna.create_study(direction='maximize')

# Ejecutar la optimización
study.optimize(objective, n_trials=50)

# Imprimir los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:")
print(study.best_params)

# Evaluar el modelo con los mejores hiperparámetros en el conjunto de test
best_model = create_model(study.best_trial)
best_model.fit(
    X_train_scaled, y_train_encoded,
    epochs=200,
    batch_size=8,
    validation_data=(X_val_scaled, y_val_encoded),
    callbacks=[early_stopping],
    verbose=1
)

# Evaluar en test
test_loss, test_accuracy = best_model.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print(f"Exactitud en test con los mejores hiperparámetros: {test_accuracy:.4f}")

[I 2024-12-02 13:34:00,299] A new study created in memory with name: no-name-51bf1eaf-6c20-46f4-9fb2-7e2e784e5b55


   region  tenure  age  marital  address  income  ed  employ  retire  gender  \
0       2      13   44        1        9    64.0   4       5       0       0   
1       3      11   33        1        7   136.0   5       5       0       0   
2       3      68   52        1       24   116.0   1      29       0       1   
3       2      33   33        0       12    33.0   2       0       0       1   
4       2      23   30        1        9    30.0   1       2       0       0   

   reside  custcat  
0       2        1  
1       6        4  
2       2        3  
3       1        1  
4       4        3  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   region   1000 non-null   int64  
 1   tenure   1000 non-null   int64  
 2   age      1000 non-null   int64  
 3   marital  1000 non-null   int64  
 4   address  1000 non-null   int64  
 5   income   1000 no

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10836\1901716820.py:70: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-2)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_10836\1901716820.py:73: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
C:\Users\Administrator\Documents\Proyecto seg clientes refuerzo\myenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
 

Mejores hiperparámetros:
{'num_neurons_1': 73, 'num_neurons_2': 32, 'l2_reg': 0.00016096663046043358, 'learning_rate': 0.0010257099551825868}
Epoch 1/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3438 - loss: 1.3435 - val_accuracy: 0.3669 - val_loss: 1.3132
Epoch 2/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4386 - loss: 1.2318 - val_accuracy: 0.3905 - val_loss: 1.2946
Epoch 3/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4280 - loss: 1.2365 - val_accuracy: 0.3669 - val_loss: 1.3321
Epoch 4/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4639 - loss: 1.2150 - val_accuracy: 0.3787 - val_loss: 1.3270
Epoch 5/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4703 - loss: 1.1916 - val_accuracy: 0.3905 - val_loss: 1.3287
Epoch 6/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4540 - loss: 1.2036 - val_accuracy: 0.3787 - val_loss: 1.3210
Epoch 7/200
99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4509 - loss: 1.1986 - val_accur